In [ ]:
!pip install scikit-image matplotlib tqdm

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import blob_dog
from itertools import product
from tqdm import tqdm
from read_chroma import read_chromato_and_chromato_cube

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'

mod_time = 1.7

In [ ]:
chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
min_sigma = 2
max_sigma = 30
sigma_ratio = 1.6
thresholds = [0.001, 0.005, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 1.0]
mass_isoprene = mass =67 - 35
mass_ethanol =  46-35

# Dictionnaire pour stocker les résultats pour chaque masse et chaque seuil
results = {}


results[mass] = {}  # dictionnaire par masse

for thresh in thresholds:
    # Détection des blobs pour cette masse avec ce seuil
    blobs = blob_dog(chromato_cube_preprocessed[mass], 
                        min_sigma=min_sigma, 
                        max_sigma=max_sigma, 
                        sigma_ratio=sigma_ratio,
                        threshold=thresh)
    
    blobs = StandardScaler().fit_transform(blobs)
    # Stockage des résultats
    results[mass][thresh] = blobs
        
    print(f"Masse {mass}, Threshold = {thresh}, nombre de blobs détectés : {len(blobs)}")

# Analyse des résultats pour déterminer le seuil optimal
for mass in results:
    print(f"\nRésultats pour la masse {mass}:")
    for thresh in thresholds:
        print(f"  Threshold = {thresh}: {len(results[mass][thresh])} blobs")

# Visualisation pour une masse spécifique (par exemple, la première)
if len(results) > 0:
    mass_to_visualize = mass_isoprene  # Choisissez la masse à visualiser
    
    fig, axes = plt.subplots(1, len(thresholds), figsize=(15, 3))
    fig.suptitle(f"Détection de blobs pour masse {mass_to_visualize} avec différents seuils")
    
    for i, thresh in enumerate(thresholds):
        blobs = results[mass_to_visualize][thresh]
        
        # Affichage de l'image
        axes[i].imshow(chromato_cube_preprocessed[mass_to_visualize], cmap='gray')
        
        # Affichage des blobs détectés
        for blob in blobs:
            y, x, r = blob
            c = plt.Circle((x, y), r, color='red', linewidth=1, fill=False)
            axes[i].add_patch(c)
            
        axes[i].set_title(f"Threshold = {thresh}\n{len(blobs)} blobs")
        axes[i].set_axis_off()
    
    plt.tight_layout()
    plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import blob_dog
from skimage import exposure, filters, measure
from scipy import ndimage

# Fonction pour afficher l'image et ses caractéristiques
def analyze_image(image, title="Analyse d'image"):
    plt.figure(figsize=(12, 8))
    
    # Image originale
    plt.subplot(2, 2, 1)
    plt.imshow(image, cmap='gray')
    plt.title("Image originale")
    plt.colorbar()
    
    # Histogramme
    plt.subplot(2, 2, 2)
    plt.hist(image.flatten(), bins=50)
    plt.title(f"Histogramme\nMin: {image.min():.6f}, Max: {image.max():.6f}")
    plt.xlabel("Valeur de pixel")
    plt.ylabel("Fréquence")
    
    # Profile horizontal (moyenne des lignes)
    plt.subplot(2, 2, 3)
    horizontal_profile = np.mean(image, axis=0)
    plt.plot(horizontal_profile)
    plt.title("Profil horizontal (moyenne des lignes)")
    plt.xlabel("Position X")
    plt.ylabel("Intensité moyenne")
    
    # Profile vertical (moyenne des colonnes)
    plt.subplot(2, 2, 4)
    vertical_profile = np.mean(image, axis=1)
    plt.plot(vertical_profile)
    plt.title("Profil vertical (moyenne des colonnes)")
    plt.xlabel("Position Y")
    plt.ylabel("Intensité moyenne")
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()
    
    # Statistiques de l'image
    print(f"Statistiques pour {title}:")
    print(f"Dimensions: {image.shape}")
    print(f"Min: {image.min()}, Max: {image.max()}")
    print(f"Moyenne: {image.mean():.6f}, Écart-type: {image.std():.6f}")
    print(f"25% des valeurs sont inférieures à: {np.percentile(image, 25):.6f}")
    print(f"50% des valeurs sont inférieures à: {np.percentile(image, 50):.6f}")
    print(f"75% des valeurs sont inférieures à: {np.percentile(image, 75):.6f}")
    print(f"99% des valeurs sont inférieures à: {np.percentile(image, 99):.6f}")
    
    if np.allclose(image, image.astype(int)):
        print("L'image contient uniquement des valeurs entières")
    else:
        print("L'image contient des valeurs décimales")
        
    # Vérifier si l'image a des valeurs principalement binaires
    unique_vals = np.unique(image)
    if len(unique_vals) <= 2:
        print(f"ATTENTION: L'image semble être binaire avec seulement {len(unique_vals)} valeurs uniques!")
    elif len(unique_vals) < 10:
        print(f"ATTENTION: L'image a peu de valeurs uniques: {len(unique_vals)}")
        print(f"Valeurs uniques: {unique_vals}")

# Masse à analyser
mass = 32

# Récupérer l'image pour cette masse
image = chromato_cube_preprocessed[mass].copy()

# Analyse de l'image originale
analyze_image(image, f"Image originale (masse {mass})")

# --------- ESSAI 1: Normalisation des données ---------
# Normaliser les valeurs de l'image entre 0 et 1
image_norm = (image - image.min()) / (image.max() - image.min())
analyze_image(image_norm, f"Image normalisée (masse {mass})")

# --------- ESSAI 2: Appliquer un filtre gaussien pour réduire le bruit ---------
image_smooth = filters.gaussian(image_norm, sigma=1)
analyze_image(image_smooth, f"Image normalisée + lissée (masse {mass})")

# --------- ESSAI 3: Amélioration du contraste ---------
image_eq = exposure.equalize_hist(image_norm)
analyze_image(image_eq, f"Image à contraste amélioré (masse {mass})")

# --------- ESSAI 4: Utiliser une autre méthode de détection ---------
# Au lieu de blob_dog, essayons une approche basée sur le seuillage et l'étiquetage
# Nous prenons l'image à contraste amélioré
threshold_value = filters.threshold_otsu(image_eq)
binary_image = image_eq > threshold_value
analyze_image(binary_image, f"Image binarisée (masse {mass})")

# Étiquetage des composantes connexes
labeled_image, num_blobs = ndimage.label(binary_image)
print(f"Nombre de blobs détectés par étiquetage: {num_blobs}")

# Visualisation des résultats
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(image_eq, cmap='gray')
plt.title("Image à contraste amélioré")

plt.subplot(1, 2, 2)
plt.imshow(labeled_image, cmap='nipy_spectral')
plt.title(f"Étiquetage des blobs: {num_blobs} trouvés")
plt.colorbar()
plt.tight_layout()
plt.show()

# --------- ESSAI 5: Testons blob_dog avec différents prétraitements ---------
# Testons blob_dog pour voir comment il réagit aux différentes images prétraitées
test_thresholds = [0.01, 0.1, 0.5]
test_images = {
    'Originale': image,
    'Normalisée': image_norm,
    'Lissée': image_smooth,
    'Contraste amélioré': image_eq
}

min_sigma = 2
max_sigma = 30
sigma_ratio = 1.6

results = {}

for img_name, img_data in test_images.items():
    results[img_name] = {}
    print(f"\nRésultats pour l'image {img_name}:")
    
    for thresh in test_thresholds:
        blobs = blob_dog(img_data, 
                        min_sigma=min_sigma, 
                        max_sigma=max_sigma, 
                        sigma_ratio=sigma_ratio,
                        threshold=thresh)
        
        results[img_name][thresh] = blobs
        print(f"  Threshold = {thresh}: {len(blobs)} blobs")

# Visualisation des résultats pour une comparaison complète
fig, axes = plt.subplots(len(test_images), len(test_thresholds), figsize=(15, 15))
fig.suptitle(f"Comparaison des méthodes de prétraitement et seuils pour masse {mass}")

for i, (img_name, img_data) in enumerate(test_images.items()):
    for j, thresh in enumerate(test_thresholds):
        ax = axes[i, j]
        blobs = results[img_name][thresh]
        
        # Affichage de l'image
        ax.imshow(img_data, cmap='gray')
        
        # Affichage des blobs détectés
        for blob in blobs:
            y, x, r = blob
            c = plt.Circle((x, y), r, color='red', linewidth=1, fill=False)
            ax.add_patch(c)
            
        ax.set_title(f"{img_name}\nThreshold = {thresh}\n{len(blobs)} blobs")
        ax.set_axis_off()

plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()

# --------- ESSAI 6: Implémentation alternative de la détection de blobs ---------
# Utilisons une approche basée sur les extrema locaux
from skimage.feature import peak_local_max

# Fonction pour détecter les extrema locaux
def detect_local_maxima(image, min_distance=5, threshold_abs=0.1, threshold_rel=0.2):
    # Détection des maxima locaux
    coordinates = peak_local_max(image, min_distance=min_distance, 
                                threshold_abs=threshold_abs,
                                threshold_rel=threshold_rel)
    return coordinates

# Tests avec différents paramètres
test_rel_thresholds = [0.1, 0.3, 0.5]
local_maxima_results = {}

for thresh_rel in test_rel_thresholds:
    # Utilisons l'image à contraste amélioré
    coordinates = detect_local_maxima(image_eq, 
                                    min_distance=5, 
                                    threshold_abs=0.01,
                                    threshold_rel=thresh_rel)
    
    local_maxima_results[thresh_rel] = coordinates
    print(f"Méthode des maxima locaux, threshold_rel = {thresh_rel}: {len(coordinates)} points détectés")

# Visualisation des résultats
fig, axes = plt.subplots(1, len(test_rel_thresholds), figsize=(15, 5))
fig.suptitle(f"Détection des maxima locaux pour masse {mass}")

for i, thresh_rel in enumerate(test_rel_thresholds):
    ax = axes[i]
    coordinates = local_maxima_results[thresh_rel]
    
    # Affichage de l'image
    ax.imshow(image_eq, cmap='gray')
    
    # Affichage des points détectés
    ax.plot(coordinates[:, 1], coordinates[:, 0], 'r.', markersize=5)
            
    ax.set_title(f"threshold_rel = {thresh_rel}\n{len(coordinates)} points")
    ax.set_axis_off()

plt.tight_layout()
plt.show()

# Recommandations finales
print("\n--- CONCLUSIONS ET RECOMMANDATIONS ---")
print("1. Analysez la distribution des valeurs dans vos images")
print("2. Essayez la méthode alternative d'étiquetage des composantes connexes si blob_dog ne fonctionne pas")
print("3. Utilisez la détection des maxima locaux pour une approche plus simple")
print("4. Considérez un prétraitement d'égalisation d'histogramme pour améliorer le contraste")

In [ ]:


# Paramètres à tester
min_sigma_list = [1, 2, 3]
max_sigma_list = [10, 20]
threshold_list = [0.01, 0.03, 0.05]
masse_isoprene = mass = 67


results = []

# Grid Search
for min_sigma, max_sigma, threshold in tqdm(product(min_sigma_list, max_sigma_list, threshold_list),
                                            total=len(min_sigma_list)*len(max_sigma_list)*len(threshold_list)):
    
    blobs = blob_dog(chromato_cube_preprocessed[mass],
                    min_sigma=min_sigma,
                    max_sigma=max_sigma,
                    threshold=threshold,
                    overlap=0.5)

    score = len(blobs)  # Critère simple : plus de blobs = plus de pics détectés

    results.append({
        'min_sigma': min_sigma,
        'max_sigma': max_sigma,
        'threshold': threshold,
        'score': score,
        'blobs': blobs
    })

# Tri des meilleurs paramètres (par exemple : le plus de blobs)
best_result = sorted(results, key=lambda x: x['score'], reverse=True)[0]

print("\n✅ Best Parameters Found:")
print(f"min_sigma = {best_result['min_sigma']}")
print(f"max_sigma = {best_result['max_sigma']}")
print(f"threshold = {best_result['threshold']}")
print(f"Nb blobs detected = {best_result['score']}")

# Affichage des blobs détectés
fig, ax = plt.subplots()
ax.imshow(file_path, cmap='gray')
for blob in best_result['blobs']:
    y, x, r = blob
    c = plt.Circle((x, y), r * np.sqrt(2), color='red', linewidth=1.5, fill=False)
    ax.add_patch(c)
plt.title("Best blob_dog parameters")
plt.show()



overlap: par defaut = 0.5
sert à supprimer les blobs redondants qui se chevauchent trop.
Lorsque plusieurs blobs détectés se recouvrent de plus de overlap (par défaut 0.5), seul le plus significatif est gardé (en général celui avec le rayon le plus grand ou la valeur de réponse la plus élevée).


In [ ]:
min_sigma = 1
max_sigma = 30
sigma_ratio = 1.6
threshold_abs = 0.01

In [ ]:
from skimage.feature import blob_log
import matplotlib.pyplot as plt

#overlap
def test_overlap(chromato_2d, min_sigma, max_sigma, num_sigma, threshold_abs):
    overlaps = [0.1, 0.3, 0.5, 0.7, 0.9]
    results = {}

    for ov in overlaps:
        blobs = blob_log(
            chromato_2d,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            num_sigma=num_sigma,
            threshold=threshold_abs,
            overlap=ov
        )
        results[ov] = blobs
        print(f"Overlap {ov} : {len(blobs)} blobs")

    return results